Created on Tuesday 05 January 2021  

**Group 2 - Identification**

**CNIL scraper**

@authors: Marianne Manson

# CNIL

Article link: https://www.cnil.fr/fr/les-collectivites-territoriales-et-lopen-data-concilier-ouverture-des-donnees-et-protection-des

Article list (06/01/2021) : from https://www.cnil.fr/fr/actualite to https://www.cnil.fr/fr/actualite?page=32

Warning : https://www.cnil.fr/fr/actualite?page=1 is not the same as the first page

In [2]:
#import de lib
import pandas as pd
import numpy as np
import re
from requests import get
from bs4 import BeautifulSoup

In [7]:
df_scrap = pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df_scrap

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [41]:
def get_url_page_cnil(first_page):
    """Documentation
    
    Parameters:
        first_page: url of first the page where articles are listed
    
    Out:
        url_page_list: list with the urls of the pages where articles are listed
    """
    url_page_list = [first_page]
    req = get(first_page)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    try:
        load_more_zone = html_soup.find("li",{'class':"pager-next first last"})
        next_url = "https://www.cnil.fr" + load_more_zone.find("a")['href']
        #print(next_url)
        url_page_list.extend(get_url_page_cnil(next_url))
    except:
        pass
    return url_page_list

In [42]:
test_page_list = get_url_page_cnil("https://www.cnil.fr/fr/actualite")
test_page_list

['https://www.cnil.fr/fr/actualite',
 'https://www.cnil.fr/fr/actualite?page=1',
 'https://www.cnil.fr/fr/actualite?page=2',
 'https://www.cnil.fr/fr/actualite?page=3',
 'https://www.cnil.fr/fr/actualite?page=4',
 'https://www.cnil.fr/fr/actualite?page=5',
 'https://www.cnil.fr/fr/actualite?page=6',
 'https://www.cnil.fr/fr/actualite?page=7',
 'https://www.cnil.fr/fr/actualite?page=8',
 'https://www.cnil.fr/fr/actualite?page=9',
 'https://www.cnil.fr/fr/actualite?page=10',
 'https://www.cnil.fr/fr/actualite?page=11',
 'https://www.cnil.fr/fr/actualite?page=12',
 'https://www.cnil.fr/fr/actualite?page=13',
 'https://www.cnil.fr/fr/actualite?page=14',
 'https://www.cnil.fr/fr/actualite?page=15',
 'https://www.cnil.fr/fr/actualite?page=16',
 'https://www.cnil.fr/fr/actualite?page=17',
 'https://www.cnil.fr/fr/actualite?page=18',
 'https://www.cnil.fr/fr/actualite?page=19',
 'https://www.cnil.fr/fr/actualite?page=20',
 'https://www.cnil.fr/fr/actualite?page=21',
 'https://www.cnil.fr/fr/ac

In [26]:
def get_url_article_cnil(page_url):
    """Documentation
    
    Parameters:
        page_url: url of the page where articles are listed
    
    Out:
        url_art_list: list with the urls of public articles
    """
    url_art_list = []
    req = get(page_url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    art_divs = html_soup.find_all("div",{'class':"row liste"})
    for div in art_divs:
        zone_link = div.find("h3")
        url = "https://www.cnil.fr" + zone_link.find("a")['href']
        url_art_list.append(url)
    return url_art_list

In [27]:
test_list = get_url_article_cnil("https://www.cnil.fr/fr/actualite")
test_list

['https://www.cnil.fr/fr/prospection-commerciale-sanction-de-20-000-euros-lencontre-de-la-societe-nestor',
 'https://www.cnil.fr/fr/la-collecte-de-donnees-dans-le-cadre-de-la-vaccination-contre-la-covid-19-quelles-garanties-pour-les',
 'https://www.cnil.fr/fr/brexit-le-rgpd-reste-applicable-au-royaume-uni-jusquau-1er-juillet-2021',
 'https://www.cnil.fr/fr/violations-de-donnees-de-sante-la-cnil-sanctionne-deux-medecins',
 'https://www.cnil.fr/fr/effacer-ses-donnees-dun-ordinateur-dun-telephone-ou-dune-tablette-avant-de-sen-separer',
 'https://www.cnil.fr/fr/la-cnil-salue-laction-de-supdpo-le-reseau-des-dpo-de-lenseignement-superieur-de-la-recherche']

In [51]:
def scraping(url):
    '''Documentation
    
    Parameters:
        url: url of the scraped page
    
    Out:
        new_row: data to put in dataframe
    '''
    req = get(url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    #content, content_html
    try:
        content_html = html_soup.find("div",{'class':"field-item even"})
        content = content_html.text
    except:
        #maybe find a way to take into account multiple article structures instead
        content_html = np.nan
        content = np.nan
    #date
    date = html_soup.find("div",{'class':"ctn-gen-auteur"})
    date = date.text
    #title
    zone_title = html_soup.find("div",{'class':"ctn-gen-titre"})
    title = zone_title.find("h1")
    title = title.text
    #img
    try:
        zone_img = html_soup.find("div",{'class':"ctn-gen-visuel"})
        img = zone_img.find("img")['src']
    except:
        img = "no_data"
    #tag
    zone_tag = html_soup.find("div",{'class':"mots cles"})
    try:
        tags_li_list = zone_tag.find_all("li")
        tags_list = []
        for tag in tags_li_list:
            tags_list.append(tag.text[1:]) #[1:] to remove "#"
    except:
        tags_list = "no_data"
    # add data to dataframe 
    new_row = {'art_content': content ,
               'art_content_html': content_html ,
               'art_extract_datetime': date ,
               'art_lang': 'fr' , 
               'art_title' : title , 
               'art_url' : url ,
               'src_name' :'cnil'  ,
               'src_type' : 'xpath_source' ,
               'src_url' : 'https://www.cnil.fr/',
               'src_img' : img ,
               'art_auth': "no_data", # No author specified
               'art_tag': tags_list}
    return new_row

In [53]:
new_row = scraping("https://www.cnil.fr/fr/video-le-youtubeur-cookie-connecte-repond-vos-questions-sur-larrivee-du-rgpd")
df_test = df_scrap.append(new_row, ignore_index=True)
df_test

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,Le Youtubeur « Cookie Connecté » a collaboré a...,[[Le Youtubeur « Cookie Connecté » a collaboré...,06 février 2018,fr,VIDEO | Le youtubeur Cookie Connecté répond à ...,https://www.cnil.fr/fr/video-le-youtubeur-cook...,cnil,xpath_source,https://www.cnil.fr/,https://www.cnil.fr/sites/default/files/styles...,no_data,[Règlement européen]


In [54]:
#Test on all pages
df_test = df_scrap
page_list = get_url_page_cnil("https://www.cnil.fr/fr/actualite")
article_list = []
for page in page_list:
    article_list.extend(get_url_article_cnil(page))
for url in article_list:
    #print(url)
    new_row = scraping(url)
    df_test = df_test.append(new_row, ignore_index=True)
df_test

https://www.cnil.fr/fr/prospection-commerciale-sanction-de-20-000-euros-lencontre-de-la-societe-nestor
https://www.cnil.fr/fr/la-collecte-de-donnees-dans-le-cadre-de-la-vaccination-contre-la-covid-19-quelles-garanties-pour-les
https://www.cnil.fr/fr/brexit-le-rgpd-reste-applicable-au-royaume-uni-jusquau-1er-juillet-2021
https://www.cnil.fr/fr/violations-de-donnees-de-sante-la-cnil-sanctionne-deux-medecins
https://www.cnil.fr/fr/effacer-ses-donnees-dun-ordinateur-dun-telephone-ou-dune-tablette-avant-de-sen-separer
https://www.cnil.fr/fr/la-cnil-salue-laction-de-supdpo-le-reseau-des-dpo-de-lenseignement-superieur-de-la-recherche
https://www.cnil.fr/fr/publication-des-decrets-relatifs-aux-fichiers-pasp-gipasp-et-easp-la-cnil-precise-sa-mission
https://www.cnil.fr/fr/cookies-sanction-de-35-millions-deuros-lencontre-damazon-europe-core
https://www.cnil.fr/fr/cookies-sanction-de-60-millions-deuros-lencontre-de-google-llc-et-de-40-millions-deuros-lencontre-de
https://www.cnil.fr/fr/certificat

https://www.cnil.fr/fr/la-cnil-publie-son-registre-rgpd
https://www.cnil.fr/fr/traitements-de-donnees-de-sante-comment-faire-la-distinction-entre-un-entrepot-et-une-recherche-et
https://www.cnil.fr/fr/robots-connectes-et-donnees-personnelles-les-conseils-de-la-cnil
https://www.cnil.fr/fr/televiseurs-connectes-les-conseils-de-la-cnil
https://www.cnil.fr/fr/rencontrez-la-cnil-au-forum-de-lemploi-tech-de-letat-le-12-decembre-2019
https://www.cnil.fr/fr/la-cnil-lance-une-consultation-publique-sur-le-referentiel-relatif-la-designation-des-conducteurs
https://www.cnil.fr/fr/liste-traitements-aipd-non-requise
https://www.cnil.fr/fr/open-data-la-cnil-et-la-cada-publient-un-guide-pratique-de-la-publication-en-ligne-et-de-la
https://www.cnil.fr/fr/jeux-sur-votre-smartphone-quand-cest-gratuit
https://www.cnil.fr/fr/collectivites-territoriales-la-cnil-publie-un-guide-de-sensibilisation-au-rgpd
https://www.cnil.fr/fr/lenregistrement-video-ou-la-capture-decran-couple-lenregistrement-des-conversation

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,"La CNIL, saisie de plusieurs plaintes, a effec...","[[La CNIL, saisie de plusieurs plaintes, a eff...",05 janvier 2021,fr,Prospection commerciale : sanction de 20 000 e...,https://www.cnil.fr/fr/prospection-commerciale...,cnil,xpath_source,https://www.cnil.fr/,no_data,no_data,"[Sanctions, Prospection, Consentement, Informa..."
1,Saisie par le ministre des solidarités et de l...,[[Saisie par le ministre des solidarités et de...,30 décembre 2020,fr,La collecte de données dans le cadre de la vac...,https://www.cnil.fr/fr/la-collecte-de-donnees-...,cnil,xpath_source,https://www.cnil.fr/,https://www.cnil.fr/sites/default/files/styles...,no_data,"[COVID-19, Santé, Avis]"
2,Dans le cadre de l’accord de commerce et de co...,[[Dans le cadre de l’accord de commerce et de ...,28 décembre 2020,fr,Brexit : le RGPD reste applicable au Royaume-U...,https://www.cnil.fr/fr/brexit-le-rgpd-reste-ap...,cnil,xpath_source,https://www.cnil.fr/,no_data,no_data,[Brexit]
3,À la suite d’un contrôle en ligne réalisé en s...,[[À la suite d’un contrôle en ligne réalisé en...,17 décembre 2020,fr,Violations de données de santé : la CNIL sanct...,https://www.cnil.fr/fr/violations-de-donnees-d...,cnil,xpath_source,https://www.cnil.fr/,no_data,no_data,"[Sanctions, Sécurité informatique, Données de ..."
4,Votre disque dur ou la mémoire de votre appare...,[[Votre disque dur ou la mémoire de votre appa...,16 décembre 2020,fr,"Effacer ses données d’un ordinateur, d’un télé...",https://www.cnil.fr/fr/effacer-ses-donnees-dun...,cnil,xpath_source,https://www.cnil.fr/,https://www.cnil.fr/sites/default/files/styles...,no_data,"[Sécurité informatique, Les essentiels]"
...,...,...,...,...,...,...,...,...,...,...,...,...
115,La CPU et la CNIL entendent notamment accompag...,[[La CPU et la CNIL entendent notamment accomp...,30 janvier 2019,fr,La CNIL et la Conférence des présidents d’univ...,https://www.cnil.fr/fr/la-cnil-et-la-conferenc...,cnil,xpath_source,https://www.cnil.fr/,no_data,no_data,"[Vie de la CNIL, Partenariat]"
116,Quels sont vos droits ? \nLa palette de droits...,"[[[Quels sont vos droits ? ]], \n, [La palette...",28 janvier 2019,fr,"Vous avez des droits sur vos données, profitez...",https://www.cnil.fr/fr/vous-avez-des-droits-su...,cnil,xpath_source,https://www.cnil.fr/,https://www.cnil.fr/sites/default/files/styles...,no_data,[Droits]
117,La dématérialisation des services publics cons...,[[La dématérialisation des services publics co...,23 janvier 2019,fr,Travailleurs sociaux : un kit d’information po...,https://www.cnil.fr/fr/travailleurs-sociaux-ki...,cnil,xpath_source,https://www.cnil.fr/,https://www.cnil.fr/sites/default/files/styles...,no_data,"[Social, Vie associative]"
118,Le design des interfaces des services numériqu...,[[Le design des interfaces des services numéri...,18 janvier 2019,fr,Publication du 6ème cahier Innovation & Prospe...,https://www.cnil.fr/fr/publication-du-6eme-cah...,cnil,xpath_source,https://www.cnil.fr/,https://www.cnil.fr/sites/default/files/styles...,no_data,[Innovation et prospective]


### Abstract

In [13]:
df_abstract = pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag','abstract']  )
df_abstract

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,abstract


In [61]:
def scraping_abstract(url):
    '''Documentation
    
    Parameters:
        url: url of the scraped article page
    
    Out:
        new_row: data to put in dataframe (including the abstract)
    '''
    new_row = scraping(url)
    req = get(url)
    html_soup = BeautifulSoup(req.text, 'html.parser')
    try:
        abstract_zone = html_soup.find("div",{'class':"ctn-gen-introduction"})
        abstract = abstract_zone.find("p").text
    except:
        abstract = "no_data"
    new_row['abstract'] = abstract
    return new_row
    

In [62]:
#Test on only one article
df_test = df_abstract
new_row = scraping_abstract("https://www.cnil.fr/fr/les-collectivites-territoriales-et-lopen-data-concilier-ouverture-des-donnees-et-protection-des")
df_test = df_scrap.append(new_row, ignore_index=True)
df_test

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,abstract
0,Quelles sont les obligations de publication ?\...,[[Quelles sont les obligations de publication ...,18 septembre 2019,fr,Les collectivités territoriales et l’open data...,https://www.cnil.fr/fr/les-collectivites-terri...,cnil,xpath_source,https://www.cnil.fr/,https://www.cnil.fr/sites/default/files/styles...,no_data,"[Collectivités territoriales, Open data, Les c...",L’ouverture des données (en anglais « open dat...
